In [1]:
import ast
import difflib
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# İki kod parçasını AST analizi için işleyecek fonksiyon
def analyze_ast(code):
    tree = ast.parse(code)
    nodes = list(ast.walk(tree))
    return nodes

# İki kod parçası arasındaki metinsel benzerliği hesaplayacak fonksiyon
def calculate_text_similarity(code1, code2):
    sequence = difflib.SequenceMatcher(None, code1, code2)
    return sequence.ratio() * 100  # Yüzde olarak

# Fuzzy logic modelini oluşturacak fonksiyon
def create_fuzzy_logic_model():
    # Giriş ve çıkış değişkenlerini oluştur
    ast_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'ast_similarity')
    text_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'text_similarity')
    overall_similarity = ctrl.Consequent(np.arange(0, 101, 1), 'overall_similarity')

    # Üyelik fonksiyonlarını tanımla
    ast_similarity.automf(3)
    text_similarity.automf(3)
    overall_similarity.automf(3)

    # Kuralları oluştur
    rule1 = ctrl.Rule(ast_similarity['poor'] | text_similarity['poor'], overall_similarity['poor'])
    rule2 = ctrl.Rule(ast_similarity['average'] | text_similarity['average'], overall_similarity['average'])
    rule3 = ctrl.Rule(ast_similarity['good'] & text_similarity['good'], overall_similarity['good'])

    # Kontrol sistemini oluştur ve simüle et
    similarity_control_system = ctrl.ControlSystem([rule1, rule2, rule3])
    similarity_simulation = ctrl.ControlSystemSimulation(similarity_control_system)

    return similarity_simulation

# İki kod parçasının benzerliğini hesaplayacak ana fonksiyon
def compare_codes(code1, code2):
    ast_nodes1 = analyze_ast(code1)
    ast_nodes2 = analyze_ast(code2)

    # AST benzerliğini hesapla
    ast_similarity = 100 - abs(len(ast_nodes1) - len(ast_nodes2)) / max(len(ast_nodes1), len(ast_nodes2)) * 100

    # Metinsel benzerliği hesapla
    text_similarity = calculate_text_similarity(code1, code2)

    # Fuzzy logic modelini oluştur ve girdileri ver
    similarity_simulation = create_fuzzy_logic_model()
    similarity_simulation.input['ast_similarity'] = ast_similarity
    similarity_simulation.input['text_similarity'] = text_similarity

    # Hesaplama yap
    similarity_simulation.compute()

    # Genel benzerlik sonucunu al
    overall_similarity = similarity_simulation.output['overall_similarity']
    return overall_similarity

# GitHub'dan alınan kod
github_code = """
'''
import cv2
import numpy as np
import pickle
from src.utils import Park_classifier


def demostration():
   
    # defining the params
    rect_width, rect_height = 107, 48
    carp_park_positions_path = "data/source/CarParkPos"
    video_path = "data/source/carPark.mp4"

    # creating the classifier  instance which uses basic image processes to classify
    classifier = Park_classifier(carp_park_positions_path, rect_width, rect_height)

    # Implementation of the classy
    cap = cv2.VideoCapture(video_path)
    while True:

        # reading the video frame by frame
        ret, frame = cap.read()

        # check is there a retval
        if not ret:break
        
        # prosessing the frames to prepare classify
        prosessed_frame = classifier.implement_process(frame)
        
        # drawing car parks according to its status 
        denoted_image = classifier.classify(image=frame, prosessed_image = prosessed_frame)
        
        # displaying the results
        cv2.imshow("Car Park Image which drawn According to  empty or occupied", denoted_image)
        
        # exit condition
        k = cv2.waitKey(1)
        if k & 0xFF == ord('q'):
            break
        
        if k & 0xFF == ord('s'):
            cv2.imwrite("output.jpg", denoted_image)

    # re-allocating sources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    demostration()
'''
"""

# ChatGPT'nin yazdığı örnek kod
example_code = """
import cv2
import numpy as np

def detect_park_space(frame, area):
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.polylines(mask, [np.array(area, np.int32)], True, 255, 2)
    mask = cv2.fillPoly(mask, [np.array(area, np.int32)], 255)
    masked = cv2.bitwise_and(frame, frame, mask=mask)
    return np.sum(masked > 0) > 1000

cap = cv2.VideoCapture('parking_video.mp4')

park_area = [(100, 100), (150, 100), (150, 150), (100, 150)]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (800, 600))

    if detect_park_space(frame, park_area):
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 0, 255), 2)
    else:
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 255, 0), 2)

    cv2.imshow('Park Space Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
"""

# Benzerlik oranına göre genel benzerlik durumunu belirleyen fonksiyon
def classify_similarity(score):
    if score < 40:
        return "Benzer değil"
    elif 40 <= score < 55:
        return "Az benzer"
    elif 55 <= score < 70:
        return "Orta düzeyde benzer"
    elif 70 <= score < 85:
        return "Benzer"
    else:
        return "Çok benzer (Kopya)"

# Benzerlik oranını hesaplayalım ve genel benzerlik durumunu belirleyelim
similarity_score = compare_codes(example_code, github_code)
similarity_classification = classify_similarity(similarity_score)
print(f"İki kod arasındaki genel benzerlik oranı: {similarity_score:.2f}% , {similarity_classification}")

İki kod arasındaki genel benzerlik oranı: 38.23% , Benzer değil


In [2]:
import ast
import difflib
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# İki kod parçasını AST analizi için işleyecek fonksiyon
def analyze_ast(code):
    tree = ast.parse(code)
    nodes = list(ast.walk(tree))
    return nodes

# İki kod parçası arasındaki metinsel benzerliği hesaplayacak fonksiyon
def calculate_text_similarity(code1, code2):
    sequence = difflib.SequenceMatcher(None, code1, code2)
    return sequence.ratio() * 100  # Yüzde olarak

# Fuzzy logic modelini oluşturacak fonksiyon
def create_fuzzy_logic_model():
    # Giriş ve çıkış değişkenlerini oluştur
    ast_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'ast_similarity')
    text_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'text_similarity')
    overall_similarity = ctrl.Consequent(np.arange(0, 101, 1), 'overall_similarity')

    # Üyelik fonksiyonlarını tanımla
    ast_similarity.automf(3)
    text_similarity.automf(3)
    overall_similarity.automf(3)

    # Kuralları oluştur
    rule1 = ctrl.Rule(ast_similarity['poor'] | text_similarity['poor'], overall_similarity['poor'])
    rule2 = ctrl.Rule(ast_similarity['average'] | text_similarity['average'], overall_similarity['average'])
    rule3 = ctrl.Rule(ast_similarity['good'] & text_similarity['good'], overall_similarity['good'])

    # Kontrol sistemini oluştur ve simüle et
    similarity_control_system = ctrl.ControlSystem([rule1, rule2, rule3])
    similarity_simulation = ctrl.ControlSystemSimulation(similarity_control_system)

    return similarity_simulation

# İki kod parçasının benzerliğini hesaplayacak ana fonksiyon
def compare_codes(code1, code2):
    ast_nodes1 = analyze_ast(code1)
    ast_nodes2 = analyze_ast(code2)

    # AST benzerliğini hesapla
    ast_similarity = 100 - abs(len(ast_nodes1) - len(ast_nodes2)) / max(len(ast_nodes1), len(ast_nodes2)) * 100

    # Metinsel benzerliği hesapla
    text_similarity = calculate_text_similarity(code1, code2)

    # Fuzzy logic modelini oluştur ve girdileri ver
    similarity_simulation = create_fuzzy_logic_model()
    similarity_simulation.input['ast_similarity'] = ast_similarity
    similarity_simulation.input['text_similarity'] = text_similarity

    # Hesaplama yap
    similarity_simulation.compute()

    # Genel benzerlik sonucunu al
    overall_similarity = similarity_simulation.output['overall_similarity']
    return overall_similarity

# GitHub'dan alınan kod
github_code = """
'''
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import time

model=YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('parking1.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
   


area9=[(511,327),(557,388),(603,383),(549,324)]



while True:    
    ret,frame = cap.read()
    if not ret:
        break
    
    frame=cv2.resize(frame,(1020,500))

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
#    print(px)
   
    list9=[]
   

    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            cx=int(x1+x2)//2
            cy=int(y1+y2)//2

       
            results9=cv2.pointPolygonTest(np.array(area9,np.int32),((cx,cy)),False)
            if results9>=0:
               cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
               cv2.circle(frame,(cx,cy),3,(0,0,255),-1)
               list9.append(c)  
        
    a9=(len(list9))
  
    if a9==1:
        cv2.polylines(frame,[np.array(area9,np.int32)],True,(0,0,255),2)
        cv2.putText(frame,str('9'),(591,398),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
    else:
        cv2.polylines(frame,[np.array(area9,np.int32)],True,(0,255,0),2)
        cv2.putText(frame,str('9'),(591,398),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
   
    cv2.imshow("RGB", frame)

    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()
'''
"""

# ChatGPT'nin yazdığı örnek kod
example_code = """
import cv2
import numpy as np

def detect_park_space(frame, area):
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.polylines(mask, [np.array(area, np.int32)], True, 255, 2)
    mask = cv2.fillPoly(mask, [np.array(area, np.int32)], 255)
    masked = cv2.bitwise_and(frame, frame, mask=mask)
    return np.sum(masked > 0) > 1000

cap = cv2.VideoCapture('parking_video.mp4')

park_area = [(100, 100), (150, 100), (150, 150), (100, 150)]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (800, 600))

    if detect_park_space(frame, park_area):
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 0, 255), 2)
    else:
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 255, 0), 2)

    cv2.imshow('Park Space Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
"""

# Benzerlik oranına göre genel benzerlik durumunu belirleyen fonksiyon
def classify_similarity(score):
    if score < 40:
        return "Benzer değil"
    elif 40 <= score < 55:
        return "Az benzer"
    elif 55 <= score < 70:
        return "Orta düzeyde benzer"
    elif 70 <= score < 85:
        return "Benzer"
    else:
        return "Çok benzer (Kopya)"

# Benzerlik oranını hesaplayalım ve genel benzerlik durumunu belirleyelim
similarity_score = compare_codes(example_code, github_code)
similarity_classification = classify_similarity(similarity_score)
print(f"İki kod arasındaki genel benzerlik oranı: {similarity_score:.2f}% , {similarity_classification}")

İki kod arasındaki genel benzerlik oranı: 35.77% , Benzer değil


In [3]:
import ast
import difflib
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# İki kod parçasını AST analizi için işleyecek fonksiyon
def analyze_ast(code):
    tree = ast.parse(code)
    nodes = list(ast.walk(tree))
    return nodes

# İki kod parçası arasındaki metinsel benzerliği hesaplayacak fonksiyon
def calculate_text_similarity(code1, code2):
    sequence = difflib.SequenceMatcher(None, code1, code2)
    return sequence.ratio() * 100  # Yüzde olarak

# Fuzzy logic modelini oluşturacak fonksiyon
def create_fuzzy_logic_model():
    # Giriş ve çıkış değişkenlerini oluştur
    ast_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'ast_similarity')
    text_similarity = ctrl.Antecedent(np.arange(0, 101, 1), 'text_similarity')
    overall_similarity = ctrl.Consequent(np.arange(0, 101, 1), 'overall_similarity')

    # Üyelik fonksiyonlarını tanımla
    ast_similarity.automf(3)
    text_similarity.automf(3)
    overall_similarity.automf(3)

    # Kuralları oluştur
    rule1 = ctrl.Rule(ast_similarity['poor'] | text_similarity['poor'], overall_similarity['poor'])
    rule2 = ctrl.Rule(ast_similarity['average'] | text_similarity['average'], overall_similarity['average'])
    rule3 = ctrl.Rule(ast_similarity['good'] & text_similarity['good'], overall_similarity['good'])

    # Kontrol sistemini oluştur ve simüle et
    similarity_control_system = ctrl.ControlSystem([rule1, rule2, rule3])
    similarity_simulation = ctrl.ControlSystemSimulation(similarity_control_system)

    return similarity_simulation

# İki kod parçasının benzerliğini hesaplayacak ana fonksiyon
def compare_codes(code1, code2):
    ast_nodes1 = analyze_ast(code1)
    ast_nodes2 = analyze_ast(code2)

    # AST benzerliğini hesapla
    ast_similarity = 100 - abs(len(ast_nodes1) - len(ast_nodes2)) / max(len(ast_nodes1), len(ast_nodes2)) * 100

    # Metinsel benzerliği hesapla
    text_similarity = calculate_text_similarity(code1, code2)

    # Fuzzy logic modelini oluştur ve girdileri ver
    similarity_simulation = create_fuzzy_logic_model()
    similarity_simulation.input['ast_similarity'] = ast_similarity
    similarity_simulation.input['text_similarity'] = text_similarity

    # Hesaplama yap
    similarity_simulation.compute()

    # Genel benzerlik sonucunu al
    overall_similarity = similarity_simulation.output['overall_similarity']
    return overall_similarity

# GitHub'dan alınan kod
github_code = """
'''
import cv2
import matplotlib.pyplot as plt
import numpy as np

from util import get_parking_spots_bboxes, empty_or_not


def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))


mask = './mask_1920_1080.png'
video_path = './samples/parking_1920_1080_loop.mp4'


mask = cv2.imread(mask, 0)

cap = cv2.VideoCapture(video_path)

connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_components)

spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30
while ret:
    ret, frame = cap.read()

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        print([diffs[j] for j in np.argsort(diffs)][::-1])

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            spot_status = empty_or_not(spot_crop)

            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1

cap.release()
cv2.destroyAllWindows()
'''
"""

# ChatGPT'nin yazdığı örnek kod
example_code = """
import cv2
import numpy as np

def detect_park_space(frame, area):
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.polylines(mask, [np.array(area, np.int32)], True, 255, 2)
    mask = cv2.fillPoly(mask, [np.array(area, np.int32)], 255)
    masked = cv2.bitwise_and(frame, frame, mask=mask)
    return np.sum(masked > 0) > 1000

cap = cv2.VideoCapture('parking_video.mp4')

park_area = [(100, 100), (150, 100), (150, 150), (100, 150)]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (800, 600))

    if detect_park_space(frame, park_area):
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 0, 255), 2)
    else:
        cv2.polylines(frame, [np.array(park_area, np.int32)], True, (0, 255, 0), 2)

    cv2.imshow('Park Space Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
"""

# Benzerlik oranına göre genel benzerlik durumunu belirleyen fonksiyon
def classify_similarity(score):
    if score < 40:
        return "Benzer değil"
    elif 40 <= score < 55:
        return "Az benzer"
    elif 55 <= score < 70:
        return "Orta düzeyde benzer"
    elif 70 <= score < 85:
        return "Benzer"
    else:
        return "Çok benzer (Kopya)"

# Benzerlik oranını hesaplayalım ve genel benzerlik durumunu belirleyelim
similarity_score = compare_codes(example_code, github_code)
similarity_classification = classify_similarity(similarity_score)
print(f"İki kod arasındaki genel benzerlik oranı: {similarity_score:.2f}% , {similarity_classification}")    

İki kod arasındaki genel benzerlik oranı: 35.85% , Benzer değil
